In [1]:
%pip install flask

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os

ROOT_DIR = os.path.abspath(os.curdir)
print(ROOT_DIR)

c:\Workspace\Pet_Projects\cv_task


In [3]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -qr requirements.txt

# import torch
# from torchvision import transforms as T
# import torch.nn.functional as F
from PIL import Image, ImageDraw, ImageFont
# from IPython import display
# from IPython.display import clear_output
from pathlib import Path
# import yaml
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg
# import glob
import flask
import io
# import numpy as np
import base64
import time 

%matplotlib inline
#display = utils.notebook_init()  # checks

c:\Workspace\Pet_Projects\cv_task\yolov5


fatal: destination path 'yolov5' already exists and is not an empty directory.


Note: you may need to restart the kernel to use updated packages.


In [4]:
# return back to project folder
%cd $ROOT_DIR

c:\Workspace\Pet_Projects\cv_task


In [6]:
app = flask.Flask(__name__, template_folder='templates')

def get_latest_folder(path=''):
    if path:
        # Получим список имен всего содержимого папки
        # и превратим их в абсолютные пути
        dir_list = [os.path.join(path, x) for x in os.listdir(path)]

        if dir_list:
            # Создадим список из путей к файлам и дат их создания.
            date_list = [[x, os.path.getctime(x)] for x in dir_list]

            # Отсортируем список по дате создания в обратном порядке
            sort_date_list = sorted(date_list, key=lambda x: x[1], reverse=True)

            # Выведем первый элемент списка. Он и будет самым последним по дате
            return sort_date_list[0][0]

# Загрузка модели для распознавания бургер-меню

# Загрузка HTML-шаблонов
@app.route("/")
def home():
    return flask.render_template("home.html")

@app.route("/predict", methods=["POST"])
def predict():
    result_image = ''
    result_text = 'Result'
    input_filename = 'input.png'
    # Загрузка изображения
    image = flask.request.files["image"].read()
    image = Image.open(io.BytesIO(image))

    if image:
        image.save(input_filename)
        start_time = time.time()
        !python yolov5/detect.py --weights burgers_best.pt --conf 0.6 --source input.png --project runs_burgers --name detect_test --augment --line=3 --hide-conf --hide-labels
        detection_taken_time = time.time() - start_time

        with open(Path(f"{get_latest_folder('runs_burgers')}/{input_filename}"), "rb") as img_file:
            b64_string = base64.b64encode(img_file.read())
        
        result_image = b64_string.decode()
        print(result_image)
        #result_text += " time = " + str(detection_taken_time)
        
        # Вывод результата
    return flask.render_template("result.html", result_text=result_text, result_image=result_image)

if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [26/May/2023 11:46:01] "GET / HTTP/1.1" 200 -
detect: weights=['burgers_best.pt'], source=input.png, data=yolov5\data\coco128.yaml, imgsz=[640, 640], conf_thres=0.6, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=True, visualize=False, update=False, project=runs_burgers, name=detect_test, exist_ok=False, line_thickness=3, hide_labels=True, hide_conf=True, half=False, dnn=False, vid_stride=1
YOLOv5  v7.0-166-g54e9515 Python-3.10.11 torch-2.0.1+cpu CPU

Fusing layers... 
Model summary: 206 layers, 12308200 parameters, 0 gradients, 16.1 GFLOPs
image 1/1 C:\Workspace\Pet_Projects\cv_task\input.png: 640x320 1 Burgermenu, 665.0ms
Speed: 1.0ms pre-process, 665.0ms inference, 1.0ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs_burgers\detect_test27
127.0.0.1 - - [26/May/2023 11:46:15] "POST /pred

...
